In [1]:
from google.colab import files

uploaded = files.upload()  # This will open a file upload dialog


Saving best.pt to best.pt


In [2]:

!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt
!pip install gradio ultralytics


Cloning into 'yolov5'...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17271 (from 3)
Receiving objects: 100% (17274/17274), 16.13 MiB | 21.40 MiB/s, done.
Resolving deltas: 100% (11859/11859), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00

In [4]:

!rm -rf yolov5

!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5


!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17271 (from 3)
Receiving objects: 100% (17274/17274), 16.13 MiB | 20.52 MiB/s, done.
Resolving deltas: 100% (11859/11859), done.
/content/yolov5/yolov5


In [6]:
from google.colab import files

uploaded = files.upload()  # This will open a file upload dialog


Saving hubconf.py to hubconf (1).py


In [7]:
import os

print("Files in /content/:", os.listdir("/content/"))


Files in /content/: ['.config', 'yolov5', 'best.pt', 'sample_data']


In [10]:
import torch
import gradio as gr
import pandas as pd
import numpy as np
from PIL import Image


MODEL_PATH = "/content/best.pt"
model = torch.hub.load("/content/yolov5", "custom", path=MODEL_PATH, source="local")

class_names = ["RBC", "WBC", "Platelets"]

def predict(image):
    results = model(image)


    boxes = results.xyxy[0].cpu().numpy()
    confs = boxes[:, 4]
    cls_ids = boxes[:, 5].astype(int)


    processed_image = Image.fromarray(results.render()[0])


    total_detections = len(cls_ids)
    class_counts = {name: 0 for name in class_names}
    class_precision = {name: 0 for name in class_names}
    class_recall = {name: 0 for name in class_names}

    for cls_id, conf in zip(cls_ids, confs):
        class_name = class_names[cls_id]
        class_counts[class_name] += 1
        class_precision[class_name] += conf


    for key in class_precision.keys():
        if class_counts[key] > 0:
            class_precision[key] /= class_counts[key]

    df = pd.DataFrame({
        "Class": list(class_counts.keys()),
        "Detections": list(class_counts.values()),
        "Precision": list(class_precision.values()),
    })

    return processed_image, df


app = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="pil"), gr.Dataframe()],
    title="YOLOv5 Object Detection with Precision & Recall",
    description="Upload an image to detect RBC, WBC, and Platelets. Table shows precision per class."
)


app.launch(share=True)

YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 212 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f771e0e8baefe79d1c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
